In [36]:
from sklearn.datasets import fetch_20newsgroups
import nltk.data
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
import logging
from gensim.models import Word2Vec,KeyedVectors
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from nltk.stem.porter import PorterStemmer
from stemming.porter2 import stem
from sklearn import model_selection
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import Imputer
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.base import BaseEstimator,ClassifierMixin
import operator
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn import cross_validation
from sklearn.ensemble import VotingClassifier

/home/dell/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#loading the news dataset from sklearn dataset library
twenty_train = fetch_20newsgroups(subset='train',shuffle=True, random_state=42)
twenty_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=42)

# tokenize to sentences based on the notations followed in english literature
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
print("\n".join(twenty_train.data[0].split("\n")[:3]))
print(twenty_train.target_names[twenty_train.target[0]])

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
rec.autos


In [4]:
# function to parse sentences to words and remove stopwords from sentence
def sentence_to_wordlist( sentence, remove_stopwords=True ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    text = BeautifulSoup(sentence).get_text()
    #  
    # 2. Remove non-letters
    text = re.sub("[^a-zA-Z]"," ",text)
    #
    # 3. Convert words to lower case and split them
    words = text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [5]:
# function to parse text to sentences using tokenizer mentioned above
def text_to_sentences(text,tokenizer,remove_stopwords=True):
    raw_sentences = tokenizer.tokenize(text.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( sentence_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [6]:
#train_sentences and test_sentences contain the filtered sentences of training and testing data respectively

train_sentences = []  # Initialize an empty list of sentences
test_sentences = []

print "Parsing sentences from training set"
for i in range(len(twenty_train.data)):
    train_sentences += [[stem(str(word)) for word in sentence] \
                        for sentence in [text_to_sentences(twenty_train.data[i], tokenizer)]]
print len(train_sentences)
print train_sentences[0]

print "Parsing sentences from unlabeled set"
for i in range(len(twenty_test.data)):
    test_sentences += [[stem(str(word)) for word in sentence] \
                       for sentence in [text_to_sentences(twenty_test.data[i], tokenizer)]]
print len(test_sentences)
print test_sentences[0]

Parsing sentences from training set


/home/dell/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /home/dell/anaconda2/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
/home/dell/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:219: UserWarning: "." looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)
/home/dell/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:219: UserWarning: ".." looks like a filename, not markup. You shouldpro

11314
["[u'lerxst', u'wam', u'umd', u'edu', u'thing', u'subject', u'car']", "[u'nntp', u'posting', u'host', u'rac', u'wam', u'umd', u'edu', u'organization', u'university', u'maryland', u'college', u'park', u'lines', u'wondering', u'anyone', u'could', u'enlighten', u'car', u'saw', u'day']", "[u'door', u'sports', u'car', u'looked', u'late', u'early']", "[u'called', u'bricklin']", "[u'doors', u'really', u'small']", "[u'addition', u'front', u'bumper', u'separate', u'rest', u'body']", "[u'know']", "[u'anyone', u'tellme', u'model', u'name', u'engine', u'specs', u'years', u'production', u'car', u'made', u'history', u'whatever', u'info', u'funky', u'looking', u'car', u'please', u'e', u'mail']", "[u'thanks', u'il', u'brought', u'neighborhood', u'lerxst']"]
Parsing sentences from unlabeled set


/home/dell/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:219: UserWarning: "/" looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)


7532
["[u'v', u'mb', u'k', u'ubvmsd', u'cc', u'buffalo', u'edu', u'neil', u'b', u'gandler', u'subject', u'need', u'info', u'bonneville', u'organization', u'university', u'buffalo', u'lines', u'news', u'software', u'vax', u'vms', u'vnews', u'nntp', u'posting', u'host', u'ubvmsd', u'cc', u'buffalo', u'edu', u'little', u'confused', u'models', u'bonnevilles']", "[u'heard', u'le', u'se', u'lse', u'sse', u'ssei']", "[u'could', u'someone', u'tell', u'differences', u'far', u'features', u'performance']", "[u'also', u'curious', u'know', u'book', u'value', u'prefereably', u'model']", "[u'much', u'less', u'book', u'value', u'usually', u'get']", "[u'words', u'much', u'demand', u'time', u'year']", "[u'heard', u'mid', u'spring', u'early', u'summer', u'best', u'time', u'buy']", "[u'neil', u'gandler']"]


In [10]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words
# Initialize and train the model (this will take some time)

print "Training model..."
model = Word2Vec(train_sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=False)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2017-03-28 17:07:01,356 : INFO : collecting all words and their counts
2017-03-28 17:07:01,358 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-03-28 17:07:01,520 : INFO : PROGRESS: at sentence #10000, processed 166860 words, keeping 130308 word types
2017-03-28 17:07:01,546 : INFO : collected 144031 word types from a corpus of 187884 raw words and 11314 sentences
2017-03-28 17:07:01,547 : INFO : Loading a fresh vocabulary


Training model...


2017-03-28 17:07:01,636 : INFO : min_count=40 retains 28 unique words (0% of original 144031, drops 144003)
2017-03-28 17:07:01,638 : INFO : min_count=40 leaves 7150 word corpus (3% of original 187884, drops 180734)
2017-03-28 17:07:01,639 : INFO : deleting the raw counts dictionary of 144031 items
2017-03-28 17:07:01,647 : INFO : sample=0.001 downsamples 28 most-common words
2017-03-28 17:07:01,648 : INFO : downsampling leaves estimated 1012 word corpus (14.2% of prior 7150)
2017-03-28 17:07:01,650 : INFO : estimated required memory for 28 words and 300 dimensions: 81200 bytes
2017-03-28 17:07:01,651 : INFO : resetting layer weights
2017-03-28 17:07:01,654 : INFO : training model with 4 workers on 28 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=10
2017-03-28 17:07:01,655 : INFO : expecting 11314 sentences, matching count from corpus used for vocabulary survey
2017-03-28 17:07:01,894 : INFO : worker thread finished; awaiting finish of 3 more threads
2017-

In [11]:
def makeFeatureVec(words,model,num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float64")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #sorted_set = set(model.sort_vocab())
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec

In [12]:
def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
    # Print a status message every 1000th review
        if counter%1000. == 0.:
            print "Review %d of %d" % (counter, len(reviews))

    # Call the function (defined above) that makes average feature vectors
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, \
           num_features)
    # Increment the counter
        counter = counter + 1.
    return reviewFeatureVecs

In [13]:
trainDataVecs = getAvgFeatureVecs( train_sentences, model, num_features )
testDataVecs = getAvgFeatureVecs( test_sentences, model, num_features )

Review 0 of 11314
Review 1000 of 11314
Review 2000 of 11314
Review 3000 of 11314
Review 4000 of 11314
Review 5000 of 11314
Review 6000 of 11314
Review 7000 of 11314
Review 8000 of 11314
Review 9000 of 11314


/home/dell/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:23: RuntimeWarning: invalid value encountered in divide


Review 10000 of 11314
Review 11000 of 11314
Review 0 of 7532
Review 1000 of 7532
Review 2000 of 7532
Review 3000 of 7532
Review 4000 of 7532
Review 5000 of 7532
Review 6000 of 7532
Review 7000 of 7532


In [20]:
# Classification using decision tree
trainDataVecs = Imputer().fit_transform(trainDataVecs)
testDataVecs = Imputer().fit_transform(testDataVecs)
cart = DecisionTreeClassifier()
num_trees = 100
model = BaggingClassifier(base_estimator=cart, n_estimators=num_trees)
model.fit(trainDataVecs,twenty_train.target)
#results = model_selection.cross_val_score(model, trainDataVecs, twenty_train.target)
y_pred = model.predict(testDataVecs)
#print(results.mean())
print np.mean(y_pred == twenty_test.target)

0.074614976102


In [22]:
# Bagging using RandomForest Classifier 
max_features = 100
num_trees = 100
kfold = model_selection.KFold(n_splits=10)
model = RandomForestClassifier(n_estimators=num_trees, max_features=max_features)
results = model_selection.cross_val_score(model, trainDataVecs, twenty_train.target, cv=kfold)
model.fit(trainDataVecs,twenty_train.target)
y_pred = model.predict(testDataVecs)
print(results.mean())
print np.mean(y_pred==twenty_test.target)

0.101554098596
0.0763409453001


In [24]:
# Bagging using Extra trees classifier
num_trees = 100
max_features = 100
kfold = model_selection.KFold(n_splits=10 )
model = ExtraTreesClassifier(n_estimators=num_trees, max_features=max_features)
model.fit(trainDataVecs,twenty_train.target)
y_pred = model.predict(testDataVecs)
results = model_selection.cross_val_score(model, trainDataVecs, twenty_train.target, cv=kfold)
print(results.mean())
print np.mean(y_pred == twenty_test.target)

0.100846994279
0.0724907063197


In [26]:
# Boosting using Adaboost model
num_trees = 100
kfold = model_selection.KFold(n_splits=10)
model = AdaBoostClassifier(n_estimators=num_trees)
model.fit(trainDataVecs , twenty_train.target)
y_pred = model.predict(testDataVecs)
results = model_selection.cross_val_score(model, trainDataVecs, twenty_train.target, cv=kfold)
print(results.mean())
print np.mean(y_pred == twenty_test.target)

0.0875913463491
0.0734200743494


In [29]:
#stochastic gradient boosting 
num_trees = 100
kfold = model_selection.KFold(n_splits=10)
model = GradientBoostingClassifier(n_estimators=num_trees)
model.fit(trainDataVecs,twenty_train.target)
y_pred = model.predict(testDataVecs)
results = model_selection.cross_val_score(model, trainDataVecs, twenty_train.target, cv=kfold)
print(results.mean())
print np.mean(y_pred == twenty_test.target)

KeyboardInterrupt: 

In [34]:
class EnsembleClassifier(BaseEstimator, ClassifierMixin):
    """
    Ensemble classifier for scikit-learn estimators.

    Parameters
    ----------

    clf : `iterable`
      A list of scikit-learn classifier objects.
    weights : `list` (default: `None`)
      If `None`, the majority rule voting will be applied to the predicted class labels.
        If a list of weights (`float` or `int`) is provided, the averaged raw probabilities (via `predict_proba`)
        will be used to determine the most confident class label.

    """
    def __init__(self, clfs, weights=None):
        self.clfs = clfs
        self.weights = weights

    def fit(self, X, y):
        """
        Fit the scikit-learn estimators.

        Parameters
        ----------

        X : numpy array, shape = [n_samples, n_features]
            Training data
        y : list or numpy array, shape = [n_samples]
            Class labels

        """
        for clf in self.clfs:
            clf.fit(X, y)

    def predict(self, X):
        """
        Parameters
        ----------

        X : numpy array, shape = [n_samples, n_features]

        Returns
        ----------

        maj : list or numpy array, shape = [n_samples]
            Predicted class labels by majority rule

        """

        self.classes_ = np.asarray([clf.predict(X) for clf in self.clfs])
        if self.weights:
            avg = self.predict_proba(X)

            maj = np.apply_along_axis(lambda x: max(enumerate(x), key=operator.itemgetter(1))[0], axis=1, arr=avg)

        else:
            maj = np.asarray([np.argmax(np.bincount(self.classes_[:,c])) for c in range(self.classes_.shape[1])])

        return maj

    def predict_proba(self, X):

        """
        Parameters
        ----------

        X : numpy array, shape = [n_samples, n_features]

        Returns
        ----------

        avg : list or numpy array, shape = [n_samples, n_probabilities]
            Weighted average probability for each class per sample.

        """
        self.probas_ = [clf.predict_proba(X) for clf in self.clfs]
        avg = np.average(self.probas_, axis=0, weights=self.weights)

        return avg


In [40]:
clf1 = svm.SVC(probability = True)
clf2 = RandomForestClassifier()
clf3 = GaussianNB()

eclf = EnsembleClassifier(clfs=[clf1, clf2, clf3], weights=[1,1,1])

for clf, label in zip([clf1, clf2, clf3, eclf], ['SVM', 'Random Forest', 'naive Bayes', 'Ensemble']):

    scores = cross_validation.cross_val_score(clf, trainDataVecs, twenty_train.target, cv=5, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))


Accuracy: 0.05 (+/- 0.00) [SVM]
Accuracy: 0.10 (+/- 0.00) [Random Forest]
Accuracy: 0.07 (+/- 0.00) [naive Bayes]


KeyboardInterrupt: 